# Text NN classifier for Political Bias

In [1]:
%load_ext autoreload

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from sklearn.utils import shuffle
from torchsample.initializers import XavierUniform, Uniform
from torchsample.modules import ModuleTrainer
from torchsample.metrics import CategoricalAccuracy

%aimport torchsample.modules

/Users/rodrigo/Libs/torchsample/torchsample/datasets.py:16: UserWarning: Cant import nibabel.. Cant load brain images
  warnings.warn('Cant import nibabel.. Cant load brain images')


In [2]:
use_cuda = False
batch_size = 64

In [3]:
df = pd.read_csv("../data/source/newsclust.csv")
df = df.query("site != 'cbn.com'")
print(len(df))
df = df.sample(90000)

103262


Compute the bias for each one of the articles, based on the publication's known bias

In [4]:
from bias import Bias

num_classes = 7
df['bias'] = df.apply(lambda row: Bias.get_bias_for_domain(row['site']).value, axis=1)
df.head(2)

,Unnamed: 0,date,site,text,title,url,bias
72159,72159,2016-09-13T01:44:00.000+03:00,dailykos.com,35 North Carolina Sen. Richard Burr \nThe late...,Republican strategists are freaking out about ...,http://www.dailykos.com/story/2016/9/12/156932...,1
12549,12549,2016-04-13T22:50:00.000+03:00,chicagotribune.com,Lance Anderson has always worked with books. H...,"Shout Out: Lance Anderson, River Grove librari...",http://www.chicagotribune.com/suburbs/elmwood-...,2


Tokenize the text of the articles, create a vocabulary of words

In [5]:
import spacy
nlp = spacy.load('en')
nlp.pipeline = []

def tokenize_text(texts):
    docs = [doc for doc in nlp.pipe(texts, batch_size=500, n_threads=8)]
    return docs

def is_invalid_token(token):
    return token.is_punct or token.is_space or token.like_url or token.like_num or token.is_digit

def get_words_for_docs(docs):
    return [get_words_ids(doc) for doc in docs]

def get_words_ids(doc):
    return [token.orth for token in doc if not is_invalid_token(token)]

In [6]:
df['docs'] = tokenize_text(df['title'].astype(str))

In [7]:
df['words'] = get_words_for_docs(df['docs'])
df.head(1)

,Unnamed: 0,date,site,text,title,url,bias,docs,words
72159,72159,2016-09-13T01:44:00.000+03:00,dailykos.com,35 North Carolina Sen. Richard Burr \nThe late...,Republican strategists are freaking out about ...,http://www.dailykos.com/story/2016/9/12/156932...,1,"(Republican, strategists, are, freaking, out, ...","[1316, 350748, 534, 5185, 582, 581, 501, 2711,..."


Keep the most common words as vocabulary, replace the words out of vocabulary by the least frequently used word

In [8]:
from collections import Counter

vocab_size = 5000

def flatten(l):
    flat_list = [item for sublist in l for item in sublist]
    return flat_list

word_freq = Counter(flatten(df['words']))
common_words = word_freq.most_common(vocab_size)
word_freq.most_common(5)

[(504, 18219), (576246, 14844), (522, 12734), (566, 11576), (501, 11539)]

In [9]:
print(nlp.vocab.strings[501], nlp.vocab.strings[510], nlp.vocab.strings[512], nlp.vocab.strings[506])

the of and a


In [10]:
vocab = {word_id: i for (i, (word_id, freq)) in enumerate(common_words)}
print(len(vocab), vocab[501], vocab[504], vocab[510])
oov_word = vocab_size - 1
oov_word

5000 4 0 5


4999

In [11]:
def get_words_ids_if_common(words):
    return [vocab.get(word, oov_word) for word in words]

df['words_with_oov'] = df.apply(lambda row: get_words_ids_if_common(row['words']), axis=1)
df['words_with_oov'].head()

72159    [140, 4999, 101, 4999, 115, 90, 4, 4355, 38, 1...
12549    [4999, 74, 4999, 2556, 2357, 3190, 4999, 4999,...
46881    [188, 225, 4999, 34, 2126, 4999, 141, 88, 200,...
62012    [4999, 4999, 4999, 4999, 23, 1106, 1587, 358, ...
33069    [4999, 9, 736, 4999, 4999, 4999, 3237, 1939, 4...
Name: words_with_oov, dtype: object

Look at the distribution of words. The longest article has 15K words, the shortest 21.

In [12]:
lens = np.array(list(map(len, df['words'])))
(lens.max(), lens.min(), lens.mean())

(45, 0, 11.091799999999999)

Split the data for training and validation

In [13]:
TEST_DOMAINS =  {
    'bloomberg.com',
     'breitbart.com',
     'c-span.org',
     'chicagotribune.com',
     'chron.com'
}
df_train =  df.loc[~df['site'].isin(TEST_DOMAINS)]
df_test =  df.loc[df['site'].isin(TEST_DOMAINS)]
print(len(df_train))
print(len(df_test))

82312
7688


Enforce all texts to have the same length, truncating or padding if necessary

In [14]:
seq_len = 50

from keras.preprocessing import sequence

trn = sequence.pad_sequences(df_train['words_with_oov'], maxlen=seq_len, value=0)
test = sequence.pad_sequences(df_test['words_with_oov'], maxlen=seq_len, value=0)

trn_tensor = torch.from_numpy(trn).long()
test_tensor = torch.from_numpy(test).long()

trn

Using TensorFlow backend.


array([[   0,    0,    0, ...,   22,   72, 1626],
       [   0,    0,    0, ..., 4711,  116,  606],
       [   0,    0,    0, ..., 1587,  358, 4999],
       ..., 
       [   0,    0,    0, ..., 4999, 4999,    8],
       [   0,    0,    0, ...,   63, 2021, 4999],
       [   0,    0,    0, ...,   61,  668, 1411]], dtype=int32)

Prepare the tensor data for pytorch

In [15]:
labels_train_tensor = torch.from_numpy(np.array(df_train['bias']))
labels_test_tensor = torch.from_numpy(np.array(df_test['bias']))
labels_train_tensor[:3]


 1
 1
 3
[torch.LongTensor of size 3]

## Single Hidden Layer NN classifier

### Keras NN model

In [17]:
# Force use CPU
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import keras
from keras.models import Sequential, Model
from keras.layers import Embedding
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import Adam

In [38]:
k_model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(7, activation='softmax')])

In [39]:
k_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
k_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_2 (Embedding)          (None, 50, 32)        160000      embedding_input_2[0][0]          
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 1600)          0           embedding_2[0][0]                
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 100)           160100      flatten_2[0][0]                  
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 100)           0           dense_3[0][0]                    
___________________________________________________________________________________________

In [40]:
from keras.utils.np_utils import to_categorical

labels_train = np.array(df_train['bias'])
labels_test = np.array(df_test['bias'])

print(labels_test[:3])
k_model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=3, batch_size=batch_size)

[2 2 5]
Train on 82312 samples, validate on 7688 samples
Epoch 1/3
82312/82312 [==============================] - 7s - loss: 1.3330 - acc: 0.4620 - val_loss: 1.1756 - val_acc: 0.5976
Epoch 2/3
82312/82312 [==============================] - 7s - loss: 0.9994 - acc: 0.6145 - val_loss: 1.1320 - val_acc: 0.5578
Epoch 3/3
82312/82312 [==============================] - 7s - loss: 0.8962 - acc: 0.6585 - val_loss: 1.1678 - val_acc: 0.5537


Manually verify some articles. The bias for these articles should match the predicted bias below.

In [79]:
df_sample = df_test.sample(5)
df_sample[['site', 'bias']]

,site,bias
81223,bloomberg.com,2
31554,chicagotribune.com,2
43857,c-span.org,3
80184,breitbart.com,5
97101,chron.com,2


In [80]:
sample = sequence.pad_sequences(df_sample['words_with_oov'], maxlen=seq_len, value=0)
print("precision = ", np.sum(np.argmax(k_model.predict(test), axis=1) == labels_test) * 100.0 / len(labels_test))
np.argmax(k_model.predict(sample), axis=0)

precision =  55.3720083247


array([3, 1, 4, 2, 1, 3, 3])

## Site vectors

Remove the last linear classifier

In [23]:
l4 = k_model.pop()
l3 = k_model.pop()

In [24]:
k_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
k_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 50, 32)        160000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 1600)          0           embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 100)           160100      flatten_1[0][0]                  
Total params: 320,100
Trainable params: 320,100
Non-trainable params: 0
____________________________________________________________________________________________________


In [25]:
article_vectors = k_model.predict(trn)
article_vectors.shape

(82312, 100)

Generate an average vector of all the article vectors for each site

In [94]:
site_vectors = []
labels = []
sample_size = 1000
for site in df_train['site'].unique():
    site_indexes = np.where(df_train["site"] == site)[0]
    if len(site_indexes) > 0:
        site_vector = np.mean(article_vectors[site_indexes[:sample_size], :], axis=0)
        site_vectors.append(site_vector)
        labels.append(site)
site_vectors = np.array(site_vectors)
site_vectors.shape

(57, 100)

In [95]:
from sklearn.manifold import TSNE

tsne_model = TSNE(n_components=2, random_state=0)
# np.set_printoptions(suppress=True)
site_vectors_2_dim = tsne_model.fit_transform(site_vectors)
X_proj = site_vectors_2_dim[:, 0]
Y_proj = site_vectors_2_dim[:, 1]

In [96]:
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from plotly.graph_objs import Bar, Scatter, Figure, Layout, XAxis, YAxis

In [97]:
trace = Scatter(x=X_proj, y=Y_proj, mode='markers+text', text=labels, textposition='top',  marker=dict(size=10))
iplot({
    'data': [trace],
    'layout': Layout(
        #xaxis=XAxis(title='Left vs Right'), 
        #yaxis=YAxis(title='Biased vs Factual'),
        autosize=False,
        width=1000,
        height=700)},
    show_link=False
)

### PCA Analysis

In [98]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
site_vectors_pca = pca.fit(site_vectors).components_

In [99]:
trace = Scatter(x=site_vectors_pca[0], y=site_vectors_pca[1], mode='markers+text', text=labels, textposition='top',  marker=dict(size=10))
iplot({
    'data': [trace],
    'layout': Layout(
        #xaxis=XAxis(title='Left vs Right'), 
        #yaxis=YAxis(title='Biased vs Factual'),
        autosize=False,
        width=1000,
        height=700)},
    show_link=False
)